# Test random varible evaluation

For first step we create import the sufficient packages 

In [1]:
import sys
import os
import time
sys.path.append('../')
from holoclean import holocleansession , dataset , dataengine
from holoclean.utils import domainpruning
from holoclean.featurization import dcfeaturizer,dc_featurizer

In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../holoclean/mysql-connector-java-5.1.44-bin.jar")
holoclean_se=holocleansession.HolocleanSession(path)

##  Creating Dataset and dataengine and holocleansession 
In this part we create dataset for keep tracking the runnig step and dataengine for make connector to the database we try to ingest the 10 element csv file and after ingesting it returns df other wise we can read it from database

In [3]:
ds=dataset.Dataset()  
d=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',ds,holoclean_se.return_sqlcontext())
holoclean_se.set_dataengine(d)
df=d.ingest_spark('100.csv',holoclean_se.returnspark_session())

index


##  Create featurizer


In [4]:
dcCode=['t1&t2&EQ(t1.city,t2.city)&EQ(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']
#dcCode=['t1&t2&EQ(t1.city,t2.city)&GTE(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']
dcf=dcfeaturizer.DCFeaturizer(df,dcCode,d)

In this version we create new table which ha sall possible value and by using join it with original table twe create the final table.

In [5]:
dc_f=dc_featurizer.DCFeaturizer(df,dcCode,d)

we claculate the creation time in this version :

In [6]:
start_time_dd = time.time()
dc_f.table_featurizer()
finish_time_dd=time.time()
mysqlv=d.get_table_spark("dc_f1")
print(mysqlv.count())
print(mysqlv.show())

2194
+--------+------------+-------+------+--------------------+
|rv_index|assigned_val|rv_attr|tup_id|             DC_name|
+--------+------------+-------+------+--------------------+
|       3| ITE00100554|   city|    16|table1.city=table...|
|       8| ITE00100554|   city|    16|table1.city=table...|
|      18| ITE00100554|   city|    16|table1.city=table...|
|      21| ITE00100554|   city|    32|table1.city=table...|
|      23| ITE00100554|   city|    16|table1.city=table...|
|      26| ITE00100554|   city|    27|table1.city=table...|
|      26| ITE00100554|   city|    62|table1.city=table...|
|      27| ITE00100554|   city|    26|table1.city=table...|
|      28| ITE00100554|   city|    16|table1.city=table...|
|      31| ITE00100554|   city|    47|table1.city=table...|
|      32| ITE00100554|   city|    21|table1.city=table...|
|      33| ITE00100554|   city|    16|table1.city=table...|
|      36| ITE00100554|   city|    57|table1.city=table...|
|      38| ITE00100554|   city|    

In [7]:
print("--- %s seconds ---" % (finish_time_dd-start_time_dd))

--- 4.28311109543 seconds ---


In [8]:
start_time_dd = time.time()
dc_f.make_dc_f_table()
finish_time_dd=time.time()
mysqlv=d.get_table_spark("dc_f_dd")
print(mysqlv.count())
print(mysqlv.show())

2194
+--------+-------+------------+--------------------+------+
|rv_index|rv_attr|assigned_val|                  DC|tup_id|
+--------+-------+------------+--------------------+------+
|       3|   city| ITE00100554|table1.city=table...|    16|
|       8|   city| ITE00100554|table1.city=table...|    16|
|      18|   city| ITE00100554|table1.city=table...|    16|
|      21|   city| ITE00100554|table1.city=table...|    32|
|      23|   city| ITE00100554|table1.city=table...|    16|
|      26|   city| ITE00100554|table1.city=table...|    27|
|      26|   city| ITE00100554|table1.city=table...|    62|
|      27|   city| ITE00100554|table1.city=table...|    26|
|      28|   city| ITE00100554|table1.city=table...|    16|
|      31|   city| ITE00100554|table1.city=table...|    47|
|      32|   city| ITE00100554|table1.city=table...|    21|
|      33|   city| ITE00100554|table1.city=table...|    16|
|      36|   city| ITE00100554|table1.city=table...|    57|
|      38|   city| ITE00100554|tabl

Execution time is :

In [9]:
print("--- %s seconds ---" % (finish_time_dd-start_time_dd))

--- 48.0284688473 seconds ---


Because we have just one dc in all value in dc column is same